In [1]:
import os
from collections import defaultdict, Counter
from emailtome import noticeEMail
import urllib, json
import emailtome
import datetime

In [2]:
def get_user_data(event, user_data): #newest bio available from pushes, containning username,avatar url, gravatar_id, e-mail adress and real name
    user_data=defaultdict(dict)
    if 'payload' in event.keys():
        if 'shas' in event['payload'].keys() and len(event['payload']['shas']):
            if len(event['payload']['shas'][0])>=1:
                name=event['payload']['shas'][0][-1]
                e_mail=event['payload']['shas'][0][1]
        else:
            name='Nan'
            e_mail='Nan' 
    else:
            name='Nan'
            e_mail='Nan' 

    if 'actor' in event.keys(): 
        if 'id' in event['actor'].keys():
            user_data[event['actor']['id']]={'username':event['actor']['login'],
                                         'e-mail_adress': e_mail,
                                         'name':name}
    elif 'user_attributes' in event.keys():
        user_data[event['user']]
    return user_data

In [3]:
def read_aggr_file(p, directory, filename, et): #aggregate the data and generates user based json files
    pres=[]
    with open(os.path.join(directory,filename), 'r') as myfile:
        data=myfile.read()
    #print 'opened the file'
    lines=data.splitlines()
    for i in range(0, 1000):
        event=json.loads(lines[i].strip())
        #print event.keys()
        #types.add(event['type'])
        if event['type']==et:
            PRE=event
            return PRE

In [7]:
pullreq=read_aggr_file('', '', 'githubarchive_2015-12-29.json', 'PullRequestEvent')

In [18]:
pullreq['org']

{u'avatar_url': u'https://avatars.githubusercontent.com/u/6241262?',
 u'gravatar_id': u'',
 u'id': 6241262,
 u'login': u'DrupalRu',
 u'url': u'https://api.github.com/orgs/DrupalRu'}

In [22]:
membership=read_aggr_file('', '', 'githubarchive_2015-12-29.json', 'MembershipEvent')

In [23]:
membership

In [5]:
#push['payload']['forkee']['stargazers_count']
#push['payload']['forkee']['watchers_count']


AttributeError: 'NoneType' object has no attribute 'keys'

In [269]:
users_attributes=defaultdict(dict)
"""{user_id: {name: 'name', e-mail: 'e_mail', user_name:'username'}}"""
users_popularity_by_month={}
"""{user_id: {month_yeari: number_of_followers}}"""
users_projects_by_date=defaultdict(dict)
"""{user_id: {project1_id: date: count"""
projects=defaultdict(dict) 
#Brings back the following dictionary:
"""{project_id: {pull_requests_merged: {date: count},
                pull_requests_non_merged: {date: count},
                pushes: {date: count},
                forks: {date: count},
                watches: {date: count},
                stars: {date: count}"""


'{project_id: {pull_requests_merged: {date: count},\n                pull_requests_non_merged: {date: count},\n                pushes: {date: count},\n                pulls: {date: count},\n                forks: {date: count},\n                fork_apply: {date: count},\n                watches: {date: count},\n                stars: {date: count},\n                follow: {date: count}'

In [81]:
def date_from_filename(fname):
    dt = datetime.datetime.strptime(fname[len('githubarchive_'):-len('.json')], '%Y-%m-%d')
    return dt.strftime('%Y-%m')

In [80]:
def date_from_json(creation_date):
    dt = datetime.datetime.strptime(creation_date[:10], '%Y-%m-%d')
    return dt.strftime('%Y-%m')

In [41]:
def url_to_json(link):
    url = link
    response = urllib.urlopen(url)
    data = json.loads(response.read())
    return data

In [175]:
def parse(event, date_f, merging_matter):
    #print event['type']
    if type(event)==dict:
        if 'repo' in event.keys(): 
            #print'k'
            if 'id' in event['repo'].keys():
                #print 'id in repo'
                if merging_matter==False:
                    d=date_f
                    if 'user' in event.keys():
                        if event['user']['type']=='User':
                            u=event['user']['login']
                            p=event['repo']['id']
                            return (u, p, d)
                    elif 'actor' in event.keys():
                        if 'login' in event['actor'].keys():
                            u=event['actor']['login']
                            p=event['repo']['id']
                            return (u, p, d)
                        elif 'payload' in event.keys():
                            if 'actor' in event['payload'].keys():
                                u=event['payload']['actor']
                                p=event['repo']['id']
                                return (u, p, d)
                elif merging_matter==True:
                    d=date_from_json(event['payload']['pull_request']['created_at'][:10])
                    #print d
                    if 'user' in event.keys():
                        #print event.keys()
                        if event['user']['type']=='User':
                            u=event['user']['login']
                            p=event['repo']['id']
                            return (u, p, d)      
                    elif 'actor' in event.keys():
                        u=event['actor']['login']
                        p=event['repo']['id']
                        return (u, p, d)
            else:
                return ('error' ,_,_)
        elif 'repository' in event.keys():
            u=event['actor']
            p=event['repository']['id']
            d=date_f
            return (u, p, d)
        else:
            return ('error' ,_,_)
        

In [174]:
last_event

{u'actor': u'johnwtest',
 u'actor_attributes': {u'gravatar_id': u'7cf7e233fd1a5e041fd42252621eeafc',
  u'login': u'johnwtest',
  u'type': u'User'},
 u'created_at': u'2013-02-27T00:35:09-08:00',
 u'payload': {u'head': u'2c4b43654cdd4a9ca74fb4530a8418ad84631703',
  u'ref': u'refs/heads/master',
  u'shas': [[u'8fe9c142aeba4f035cffd1849b89e448374ca8d0',
    u'user1@email.org',
    u'Initial commit',
    u'First Name',
    True],
   [u'2c4b43654cdd4a9ca74fb4530a8418ad84631703',
    u'user1@email.org',
    u'This is another log message.',
    u'First Name',
    True]],
  u'size': 2},
 u'public': True,
 u'type': u'PushEvent',
 u'url': u'https://github.com/johnwtest/smallTest1/compare/9052552c85...2c4b43654c'}

In [43]:
#returns user, project, date, number of commits pushed
def parse_number_of_pushed_commits(event, date_f):
    u,p,d=parse_push(event, date_f)
    c=len(event['payload']['commits'])
    return(u,p,d,c)

In [151]:
def parse_follow_event(event):
    if type(event['actor'])==dict:
        f=event['payload']['actor']
        t=event['payload']['target']['login']
    else:
        f=event['actor']
        t=event['payload']['target']['login']
    return (f,t)

In [126]:
def parse_membership_event(event):
    u=event['actor']['login']
    o=event['org']['login']
    a=event['payload']['action']
    return (u,o,a) 

In [230]:
def parse_member_event(event):
    if 'repo' not in event.keys():
        u=event['actor']
        r=event['repository']['name']
        o=event['repository']['owner']
        a=event['payload']['action']
        return (u,r,o,a) 
    elif type(event['payload']['member'])==dict:
        u=event['payload']['member']['login']
        r=event['repo']['name']
        o=event['actor']['login']
        a=event['payload']['action']
    else:
        u=event['payload']['member']
        r=event['repo']['name']
        o=event['payload']['actor']
        a=event['payload']['action']
        return (u,r,o,a)         

In [243]:
def get_projects_dict(all_projects, user, project, date, eventt):
    if project not in all_projects.keys():
        events=defaultdict(dict)
        users=defaultdict(dict)
        dates=defaultdict(dict)
        dates[date]=1
        users[user]=dates
        events[eventt]=users
        all_projects[project]=events
        return all_projects
    elif eventt not in all_projects[project].keys():
        users=defaultdict(dict)
        dates=defaultdict(dict)
        dates[date]=1
        users[user]=dates
        all_projects[project][eventt]=users
        return all_projects 
    elif user not in all_projects[project][eventt].keys():
        dates=defaultdict(dict)
        dates[date]=1
        all_projects[project][eventt][user]=dates
        #print projects[project].keys()
        return all_projects
    elif date not in all_projects[project][eventt][user].keys():
        dates=defaultdict(dict)
        dates[date]=1
        all_projects[project][eventt][user]=dates
        return all_projects
    else:
        all_projects[project][eventt][user][date]+=1
        return all_projects

In [259]:
def get_user_activity(user_activity, user, project, date, eventt):
    if user not in user_activity.keys():
        projects=defaultdict(dict)
        events=defaultdict(dict)
        dates=defaultdict(dict)
        dates[date]=1
        events[eventt]=dates
        projects[project]=events
        user_activity[user]=projects
        return user_activity
    elif project not in user_activity[user].keys():
        events=defaultdict(dict)
        dates=defaultdict(dict)
        dates[date]=1
        events[eventt]=dates
        projects[project]=events
        user_activity[user][project]=events
        return user_activity 
    elif event not in user_activity[user][project].keys():
        dates=defaultdict(dict)
        dates[date]=1
        events[eventt]=dates
        user_activity[user][project][event]=dates
        return user_activity
    elif date not in user_activity[user][project][event].keys():
        dates=defaultdict(dict)
        dates[date]=1
        user_activity[user][project][event]=dates
        return user_activity
    else:
        user_activity[user][project][event][date]+=1
        return user_activity

In [256]:
#TEST
"""all_projects=defaultdict(dict)
p_u=((1,30, datetime.date(2015, 12, 29), 'pd'), (2,30, datetime.date(2015, 12, 30), 'pd'), (1,20, datetime.date(2015, 12, 29), 'ab'), (1,20, datetime.date(2015, 12, 29), 'ab'))
for (u, p, d, e) in p_u:
    h=get_projects_dict(all_projects, u, p, d, e)"""

In [257]:
def all_event_parser(event, date_file, follow_follower_network, all_projects, memberships, repo_memberships, users, user_activity):
    et=event['type']
    if et=='FollowEvent':
        f,t=parse_follow_event(event)
        follow_follower_network.append((f,t,date_file))
        users.add(f)
        users.add(t)
    elif et=='MembershipEvent':
        u,o,a=parse_membership_event(event)
        memberships.append((u,o,a, date_file))
        users.add(u)
    elif et=='MemberEvent':
        u,r,o,a=parse_member_event(event)
        repo_memberships.append((u,r,o,a,date_file))
        users.add(u)
        users.add(o)
    elif et=='PullRequestEvent' and event['payload']['action']=='closed':
        if 'merged' in event ['payload']['pull_request']:
            if event['payload']['pull_request']['merged']==True:
                user, project, date=parse(event, date_file, True)
                all_projects=get_projects_dict(all_projects, user, project, date, u'Closed_PullRequestEvent')
                user_activitiy=get_user_activity(user_activity, user, project, date, et)
                #return all_projects, follow_follower_network, memberships, repo_memberships, users
    elif et not in ['GistEvent','GollumEvent','CreateEvent','DeleteEvent','MemberEvent']:
        user, project, date=parse(event, date_file, False)
        if user!='error':
            all_projects=get_projects_dict(all_projects, user, project, date, et)
            user_activity=get_user_activity(user_activity, user, project, date, et)
    return all_projects, follow_follower_network, memberships, repo_memberships, users,user_activity

In [255]:
user_activtiy=defaultdict(dict)

In [260]:
all_projects=defaultdict(dict)
user_activity=defaultdict(dict)
follow_follower_network=[]
memberships=[]
repo_memberships=[]
users=set()
path=os.getcwd()
files=sorted([i for i in os.listdir(path) if i.startswith("githubarchive")])
ve=0
for f in files[0:1]:
    print f
    with open(f, 'r') as myfile:
        data=myfile.read()
        date_file=date_from_filename(f)
        lines=data.splitlines()
        print len(lines)
        for i in range(0,len(lines)):
            last_line=lines[i]
            if i%100000==0:
                print len(lines)-i
            try:
                event=json.loads(unicode(lines[i], errors='ignore').strip())
                last_event=event
                all_projects,follow_follower_network, memberships, repo_memberships,users, user_activity=all_event_parser(event, date_file, follow_follower_network, all_projects, memberships, repo_memberships, users, user_activity)
            except ValueError:
                ve+=1
                print ve
                all_projects,follow_follower_network, memberships, repo_memberships, users, user_activity=all_event_parser(event, date_file, follow_follower_network, all_projects, memberships, repo_memberships, users, user_activity)

githubarchive_2011-02-12.json
32197
32197


UnboundLocalError: local variable 'projects' referenced before assignment